# __Hyperparameter Tuning__
- Hyperparameter tuning is the process of systematically searching for the best combination of hyperparameter values for a machine learning model.
- It involves selecting a subset of hyperparameters and exploring different values for each hyperparameter to find the configuration that optimizes the model's performance on a given dataset.


## Steps to Be Followed:
1. Importing the required libraries
2. Loading the dataset
3. Analyzing the target distribution
4. Splitting the data into train and test
5. Initializing the hyperparameter space

### Step 1: Importing the Required Libraries

- Importing libraries: It imports necessary libraries for data manipulation, evaluation metrics, cross-validation, and implementation of a multilayer perceptron (MLP) classifier.

- Loading the dataset: It loads the breast cancer dataset, which contains information about breast cancer tumors and their labels.

- Importing evaluation metrics: It imports metrics like accuracy and ROC-AUC scores, which are commonly used for assessing classification model performance.

- Importing MLPClassifier: It imports the MLPClassifier class, which is used to implement a neural network classifier for the breast cancer dataset.

In [ ]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neural_network import MLPClassifier

- It uses **hyperopt** library to perform hyperparameter optimization. It defines a search space for hyperparameters and uses the Tree of Parzen Estimators (TPE) algorithm to optimize these parameters.

In [ ]:
from hyperopt import hp, fmin
from hyperopt import tpe

### Step 2: Loading the Dataset
- In this code, the breast cancer dataset is loaded using the **load_breast_cancer** function.
- The features are stored in the **breast_cancer_X** variable, and the corresponding labels are stored in **breast_cancer_y**.
- The features are then converted into a pandas DataFrame X, and the labels are converted into a Pandas Series y, where the class labels are mapped to 1 and 0.
-  **X.head()** is used to display the first few rows of the DataFrame X.

In [ ]:
# load dataset
breast_cancer_X, breast_cancer_y = load_breast_cancer(return_X_y=True)
X = pd.DataFrame(breast_cancer_X)
y = pd.Series(breast_cancer_y).map({0:1, 1:0})

X.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


**Observation**

- The output represents a portion of a tabular dataset with 5 rows and 30 columns.
- Each row corresponds to a sample or observation, and each column represents a different feature or attribute of the data.
- The values in the cells represent the measurements or values associated with each feature for a specific sample.

### Step 3: Analyzing the Target Distribution
- It calculates the frequency count of each unique value in the target variable **y** and divides it by the total length of **y**.
- This expression provides the proportion or percentage of occurrences of each unique value in the target variable, giving an understanding of the distribution or imbalance between the different classes in the dataset.

In [ ]:
y.value_counts () / len(y)



0    0.627417
1    0.372583
Name: count, dtype: float64

**Observation**

- The result is that approximately 62.74% of the target values are labeled as 0, while approximately 37.26% are labeled as 1.

### Step 4: Splitting the Data into Train and Test
- It splits the dataset into training and testing sets using a test size of 30% and prints the shapes of the training and testing data.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0)

X_train.shape, X_test.shape

((398, 30), (171, 30))

**Observation**
- The observation is that the training set contains 398 samples with 30 features, while the testing set contains 171 samples with the same 30 features.

### Step 5: Initializing the Hyperparameter Space
- It defines the hyperparameter space for a neural network classifier.
- It includes choices for the hidden layer sizes, activation function, solver algorithm, and initial learning rate.

- The objective function takes a dictionary of parameters as input.
- It creates a **params_dict** dictionary that maps the parameter names to their corresponding values.
- An instance of the **MLPClassifier** is created using the parameters from the **params_dict**.
- Cross-validation is performed using the created **MLPClassifier** on the training data (**X_train** and **y_train**) with 3-fold cross-validation and using the accuracy metric.
- The average accuracy score from cross-validation is calculated using the mean() function.
- The negative score is returned, as the optimization algorithm used in hyperparameter tuning aims to minimize the objective function.

In [ ]:
param_grid = {
    'hidden_layer_sizes': hp.choice('hidden_layer_sizes', [(10), (10, 20), (10, 20, 30)]),
    'activation': hp.choice('activation', ["tanh", "relu"]),
    'solver': hp.choice('solver', ["adam", "sgd"]),
    'learning_rate_init': hp.loguniform('learning_rate_init', np.log(0.001), np.log(1)),
}

In [ ]:
def objective(params):

    params_dict = {
        'hidden_layer_sizes': params['hidden_layer_sizes'],
        'activation': params['activation'],
        'solver': params['solver'],
        'learning_rate_init': params['learning_rate_init'],
        'random_state': 1000,
        'max_iter': 100,
    }


    mlp = MLPClassifier(**params_dict)


    score = cross_val_score(mlp, X_train, y_train,
                            scoring='accuracy', cv=3, n_jobs=4).mean()


    return -score


- It performs a hyperparameter search using the Tree of Parzen Estimators (TPE) algorithm with a maximum of 50 evaluations.

In [ ]:
search = fmin(
    fn = objective,
    space = param_grid,
    max_evals = 50,

    algo = tpe.suggest)


100%|██████████| 50/50 [00:17<00:00,  2.92trial/s, best loss: -0.9246411483253588]


In [ ]:
search

{'activation': 1,
 'hidden_layer_sizes': 1,
 'learning_rate_init': 0.007277808606690794,
 'solver': 0}

**Observation**

- It indicates the progress of the hyperparameter search using the TPE algorithm. It shows that 50 iterations (or trials) were performed, and it took approximately 13 seconds to complete. The best loss achieved during the search was -0.9246031746031745.

- The dictionary shown afterwards contains the hyperparameter configuration that resulted in the best performance. It specifies the values chosen for the **activation**, **hidden_layer_sizes**, **learning_rate_init**, and **solver** hyperparameters.